In [1]:
import ee
import geemap

In [2]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [3]:
NLCD2016 = ee.Image('USGS/NLCD/NLCD2016').select('landcover')
Map.addLayer(NLCD2016, {}, 'NLCD 2016')

In [4]:
NLCD_metadata = ee.FeatureCollection("users/giswqs/landcover/NLCD2016_metadata")
Map.addLayer(NLCD_metadata, {}, 'NLCD Metadata')

In [5]:
# point = ee.Geometry.Point([-122.4439, 37.7538])  # Sanfrancisco, CA
# point = ee.Geometry.Point([-83.9293, 36.0526])   # Knoxville, TN
point = ee.Geometry.Point([-88.3070, 41.7471])     # Chicago, IL

In [6]:
metadata = NLCD_metadata.filterBounds(point).first()
region = metadata.geometry()

In [7]:
metadata.get('2016on_bas').getInfo()

'LC08_2016256'

In [8]:
doy = metadata.get('2016on_bas').getInfo().replace('LC08_', '')
doy

'2016256'

In [9]:
ee.Date.parse('YYYYDDD', doy).format('YYYY-MM-dd').getInfo()

'2016-09-12'

In [10]:
start_date = ee.Date.parse('YYYYDDD', doy)
end_date = start_date.advance(1, 'day')

In [11]:
image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(point) \
    .filterDate(start_date, end_date) \
    .first() \
    .select('B[1-7]') \
    .clip(region)

vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-8")
Map

Map(center=[41.74710000000001, -88.307], controls=(WidgetControl(options=['position'], widget=HBox(children=(T…

In [12]:
nlcd_raw = NLCD2016.clip(region)
Map.addLayer(nlcd_raw, {}, 'NLCD')

In [13]:
raw_class_values = nlcd_raw.get('landcover_class_values').getInfo()
print(raw_class_values)

[11, 12, 21, 22, 23, 24, 31, 41, 42, 43, 51, 52, 71, 72, 73, 74, 81, 82, 90, 95]


In [14]:
n_classes = len(raw_class_values)
new_class_values = list(range(0, n_classes))
new_class_values

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [15]:
class_palette = nlcd_raw.get('landcover_class_palette').getInfo()
print(class_palette)

['476ba1', 'd1defa', 'decaca', 'd99482', 'ee0000', 'ab0000', 'b3aea3', '68ab63', '1c6330', 'b5ca8f', 'a68c30', 'ccba7d', 'e3e3c2', 'caca78', '99c247', '78ae94', 'dcd93d', 'ab7028', 'bad9eb', '70a3ba']


In [16]:
nlcd = nlcd_raw.remap(raw_class_values, new_class_values).select(['remapped'], ['landcover'])
nlcd = nlcd.set('landcover_class_values', new_class_values)
nlcd = nlcd.set('landcover_class_palette', class_palette)

In [17]:
Map.addLayer(nlcd, {}, 'NLCD')
Map

Map(center=[41.74710000000001, -88.307], controls=(WidgetControl(options=['position'], widget=HBox(children=(T…

In [18]:
# Make the training dataset.
points = nlcd.sample(**{
    'region': region,
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

Map.addLayer(points, {}, 'training', False)

In [19]:
# Use these bands for prediction.
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']

# This property of the table stores the land cover labels.
label = 'landcover'

# Overlay the points on the imagery to get training.
sample = image.select(bands).sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 30
})

# Adds a column of deterministic pseudorandom numbers. 
sample = sample.randomColumn()

split = 0.7

training = sample.filter(ee.Filter.lt('random', split))
validation = sample.filter(ee.Filter.gte('random', split))

In [20]:
training.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '2_0',
 'properties': {'B1': 72,
  'B2': 215,
  'B3': 579,
  'B4': 543,
  'B5': 2558,
  'B6': 1782,
  'B7': 1103,
  'landcover': 3,
  'random': 0.18158720869622524}}

In [21]:
validation.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '0_0',
 'properties': {'B1': 230,
  'B2': 250,
  'B3': 394,
  'B4': 326,
  'B5': 2113,
  'B6': 1338,
  'B7': 685,
  'landcover': 3,
  'random': 0.889822614157854}}

In [22]:
classifier = ee.Classifier.smileRandomForest(10).train(training, label, bands)

In [23]:
# Classify the image with the same bands used for training.
result = image.select(bands).classify(classifier)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'classfied')
Map

Map(center=[41.74710000000001, -88.307], controls=(WidgetControl(options=['position'], widget=HBox(children=(T…

In [24]:
class_values = nlcd.get('landcover_class_values').getInfo()
print(class_values)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [25]:
class_palette = nlcd.get('landcover_class_palette').getInfo()
print(class_palette)

['476ba1', 'd1defa', 'decaca', 'd99482', 'ee0000', 'ab0000', 'b3aea3', '68ab63', '1c6330', 'b5ca8f', 'a68c30', 'ccba7d', 'e3e3c2', 'caca78', '99c247', '78ae94', 'dcd93d', 'ab7028', 'bad9eb', '70a3ba']


In [26]:
landcover = result.set('classification_class_values', class_values)
landcover = landcover.set('classification_class_palette', class_palette)

In [27]:
Map.addLayer(landcover, {}, 'Land cover')
Map

Map(center=[41.74710000000001, -88.307], controls=(WidgetControl(options=['position'], widget=HBox(children=(T…

In [28]:
print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

In [29]:
Map.add_legend(builtin_legend='NLCD')
Map

Map(center=[41.74710000000001, -88.307], controls=(WidgetControl(options=['position'], widget=HBox(children=(T…

In [30]:
train_accuracy = classifier.confusionMatrix()

In [31]:
train_accuracy.getInfo()

[[280, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 156, 6, 1, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 1, 21, 0, 0],
 [2, 0, 2, 440, 4, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 0, 0],
 [0, 0, 1, 15, 197, 3, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [2, 0, 1, 3, 6, 89, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0],
 [0, 0, 0, 0, 0, 0, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 2, 2, 0, 0, 0, 192, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 2, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 29, 0, 0, 0, 0, 5, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [32]:
train_accuracy.accuracy().getInfo()

0.955954795711388

In [33]:
train_accuracy.kappa().getInfo()

0.9369213452880287

In [34]:
train_accuracy.producersAccuracy().getInfo()

[[0.9929078014184397],
 [0],
 [0.8253968253968254],
 [0.9565217391304348],
 [0.8995433789954338],
 [0.8640776699029126],
 [0.9166666666666666],
 [0.964824120603015],
 [0.5],
 [0.8181818181818182],
 [0],
 [1],
 [0.8055555555555556],
 [0],
 [0],
 [0],
 [0.8363636363636363],
 [0.9942857142857143],
 [0.8059701492537313],
 [0.75]]

In [35]:
train_accuracy.consumersAccuracy().getInfo()

[[0.975609756097561,
  0,
  0.9285714285714286,
  0.9282700421940928,
  0.9425837320574163,
  0.9468085106382979,
  0.9166666666666666,
  0.9056603773584906,
  1,
  1,
  0,
  1,
  1,
  0,
  0,
  0,
  0.989247311827957,
  0.9666666666666667,
  1,
  1]]

In [36]:
validated = validation.classify(classifier)

In [37]:
validated.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '0_0',
 'properties': {'B1': 230,
  'B2': 250,
  'B3': 394,
  'B4': 326,
  'B5': 2113,
  'B6': 1338,
  'B7': 685,
  'classification': 3,
  'landcover': 3,
  'random': 0.889822614157854}}

In [38]:
test_accuracy = validated.errorMatrix('landcover', 'classification')

In [39]:
test_accuracy.getInfo()

[[144, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 14, 25, 3, 0, 0, 9, 0, 0, 0, 0, 1, 0, 0, 0, 4, 35, 1, 0],
 [1, 0, 6, 117, 24, 3, 0, 9, 0, 0, 0, 0, 1, 0, 0, 0, 2, 31, 0, 0],
 [2, 0, 2, 31, 29, 11, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 0, 0],
 [0, 0, 1, 3, 14, 25, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0],
 [0, 0, 0, 2, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0],
 [0, 0, 5, 5, 0, 0, 0, 52, 0, 0, 0, 0, 1, 0, 0, 0, 0, 15, 6, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 5, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 7, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [40]:
test_accuracy.accuracy().getInfo()

0.7146546158812137

In [41]:
test_accuracy.producersAccuracy().getInfo()

[[0.9664429530201343],
 [0],
 [0.15217391304347827],
 [0.6030927835051546],
 [0.3258426966292135],
 [0.5434782608695652],
 [0],
 [0.6190476190476191],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0.10416666666666667],
 [0.9098984771573604],
 [0.16],
 [0]]

In [42]:
test_accuracy.consumersAccuracy().getInfo()

[[0.9664429530201343,
  0,
  0.2545454545454545,
  0.5318181818181819,
  0.3625,
  0.5952380952380952,
  0,
  0.5148514851485149,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0.25,
  0.8327526132404182,
  0.25,
  0]]

In [43]:
landcover = landcover.remap(new_class_values, raw_class_values).select(['remapped'], ['classification'])

In [44]:
landcover = landcover.set('classification_class_values', raw_class_values)
landcover = landcover.set('classification_class_palette', class_palette)

In [45]:
Map.addLayer(landcover, {}, 'Final land cover')
Map

Map(center=[41.74710000000001, -88.307], controls=(WidgetControl(options=['position'], widget=HBox(children=(T…